## Collections 
![Collections Hierarchy](./images/collections.png)

## Iterable
The `Iterable` interface allows an object to be used a *foreach* target. Defined as:

In [ ]:
public interface Iterable<T> {
    Iterator<T> iterator();

    // ...
}

In addition there are two default methods `forEach` and `spliterator`.  The focus however is the `iterator` method that returns `Iterator` - a class that provides a uniform way of accessing elements of a collection:

In [ ]:
public interface Iterator<E> {
    boolean hasNext();
    E next();
    default void remove() {
        throw new UnsupportedOperationException("remove");
    }
}

The first two  help with iteration, whereas the last one helps altering the structure of collection while iterating. Note that this interface doesn't expose methods to add element while iterating. Specific implementation of `Iterator` like `ListIterator` does provide an add method. Example:

In [ ]:
class NRandoms implements Iterable<Integer> {
    private final int COUNT;

    public NRandoms(int n) {
        this.COUNT = n;
    }

    @Override
    public Iterator<Integer> iterator() {
        return new Iterator<Integer>() {
            private int c;

            @Override
            public boolean hasNext() {
                return c < COUNT;
            }

            @Override
            public Integer next() {
                c++;
                return new Random().nextInt();
            }
        };
    }
}

for(Integer i : new NRandoms(5))
    System.out.println(i);

When structural change to a collection like adding and removing elements while iterating over the collection is required, iterator is directly used. Calling method like `remove` while iterating leads to `ConcurrentModificationException`:

In [ ]:
List<String> cities = new ArrayList<>(List.of("Beijing","Shenzhen","Hangzhou","Shanghai","Guangzhou"));
for(String city : cities) {
    if(city.startsWith("S"))
        cities.remove(city);  // throws exception
}

This exception is thrown when the iterator detects that structural modification to the collection was made not by the methods exposed by the iterator. The motivation for this behaviour is to flag that some other thread might be changing the collection while a thread is iterating over it.

## Collection
Interface contains general purpose methods that we can expect from a container of collection of elements. To **add elements** to the collection, the following methods are available:

In [ ]:
boolean add(E e);
boolean addAll(Collection<? extends E> c);

Based on the implementation, `add` can throw `UnsupportedOperationException` or throw `NullPointerException` when null elements are added. The return value indicates whether the contents of the collection changed. For example, adding duplicate element to a `Set` returns false.
The `addAll` method copies contents of one collection into another. Changing the original collecition should not impact the collection into which it was copied to.

Collection also provides facility to **remove elements** from a collection:

In [ ]:
boolean clear();
boolean remove(Object o);
boolean removeAll(Collection<?> c);
boolean retainAll(Collection<?> c);

Remove methods uilise `equals` method for comparison and return true if the collection changed after the method execution. Similar to add, `UnsupportedOperationException` or `NullPointerException` are possibility with these operations.
One notable difference with the `add` methods is that these accept `Object` instead of generic type.

To **query contents**, the following methods are available:

In [ ]:
boolean isEmpty();
int size();                           // Maximum value it can return is Integer.MAX_VALUE

boolean contains(Object o);           // Again note usage of Object instead of E
boolean containsAll(Collection<?> c);

`size` returning int can limit the number of elements in the collection in case of standard implementations like `ArrayList`, however custom implementations like *FastUtil* exist that overcome this by deprecating `size` and providing alternative method to return size. `contains` like `remove` utilises `equals`.

Collection exposes few methods to convert **collection to array**:

In [ ]:
Object[] toArray();
<T> T[] toArray(T[] t);

The second variant uses the supplied array to copy elements into (overriding contents), or returning a new one if the array is not large enough:

In [1]:
List<Integer> l1 = List.of(1,2,3,4);
Integer[] i1 = new Integer[]{-1,-2,-3,-4,-5};
l1.toArray(i1);

System.out.print(Arrays.toString(i1));

[1, 2, 3, 4, null]

In [3]:
Integer[] i2 = new Integer[]{-1,-2,-3};
l1.toArray(i2);

System.out.print(Arrays.toString(i2));

[-1, -2, -3]

In [ ]:
l1.toArray(new Integer[0]);
l1.toArray(Integer[]::new);  // same as previous

One pecularity of the method signature is that the type `T` is completely unrelated to type `E` of collection elements This shifts potential errors to runtime:

In [4]:
List.of(1,2,3).toArray(String[]::new); // compiles fine

EvalException: arraycopy: element type mismatch: can not cast one of the elements of java.lang.Object[] to the type of the destination array, java.lang.String

One possibility to fix this is to define `toArray` as `<T super E> T[] toArray(T[] a)`. But unlike `extends`, Java doesn't allow `super` in type variables. Another alternative way is to define a static method like:

In [ ]:
<A, B extends A> A[] toArray(Collection<B> c, A[] a) {
    return c.toArray(a);
}

Since generic arrays are involved, primitive arrays cannot be passed to `toArray`:

In [ ]:
int[] nums = List.of(1,2,3).toArray(new int[0]);  // compile error

## SequencedCollection
![Sequenced Collection](./images/sequenced_collections.png)  
Introduced in Java 21, `SequencedCollection` is the common class for unrelated collection classes that have a well defined order. Thus, `List`, `NavigableSet`, `Deque` have this class as a common ancestor. The class introduces the following methods:

In [ ]:
SequencedCollection<E> reversed();
// Below two methods do not return boolean unlike add. Why? Because it is modeled on
// Deque where addFirst and addLast already existed and always succeeds.
// For LinkedHashSet - what would the boolean true represent? Element was added OR element
// existed but was moved to last? It is kind of ambigous
default void addFirst(E e) {
    throw new UnsupportedOperationException();
}
default void addLast(E e) {
    throw new UnsupportedOperationException();
}
// Below two methods throws NoSuchElementException if the collection is empty
default E getFirst() {
    return this.iterator().next();
}
default E getLast() {
    return this.reversed().iterator().next();
}
// also removeFirst and removeLast

Not all the methods defined here make sense for all the child classes. For example, `NavigableSet` doesn't override the `addFirst` and `addLast` returning exception, since this set is internally sorted.

`reversed` provides a reverse order view of the collection. Any modification to the view gets transmitted to the underlying collection; the reverse may not be applicable.

In [5]:
NavigableSet<String> cities = new TreeSet<>();
cities.add("Tokyo"); cities.add("Osaka"); cities.add("Hiroshima");

NavigableSet<String> citiesReversed = cities.reversed();
citiesReversed.removeFirst();

System.out.print(cities);

[Hiroshima, Osaka]

## Set
Collection that cannot contain duplicates, it has same set of methods as collection, but has been redefined to emphasis `Set` specific behaviours. It additinally introduces `equals` and `hashCode`. As per contract, two sets are equal if they have the same size and same elements. A set determines two elements are duplicate based on *equivalence relationship* which could be:
- `equals` like in `HashSet`
- identity like in `EnumSet`
- Using custom `Comparator` or `Comparable` like in `NavigableSet`

3 different mechanisms mean you can have two elements in the same set for which `equals` are equal but `compare` is not:

In [ ]:
 class Color implements Comparable<Color> {
     public String name;
     public int r,g,b;
     public Color(String name, int r, int g, int b) {
         this.name= name;
         this.r = r;
         this.g = g;
         this.b = b;
     }
     public boolean equals(Object o) {
         if(o instanceof Color othr) {
             return this.name.equals(othr.name);
         }
         return false;
     }
     public int compareTo(Color othr) 
         return (othr.r + othr.g + othr.b) - (this.r + this.g + this.b);
    }
}

Color c1 = new Color("red", 255,0,0);
Color c2 = new Color("red", 254,0,0);
c1.equals(c2); // true

TreeSet<Color> colors = new TreeSet<>();
colors.add(c1); // true, added to set
colors.add(c2); // true, added to set

### HashSet
Is the most commonly used set implementation which internally uses a hash map implementation. Therefore the performance characteristics is same as a `HashMap` - meaning for lightly loaded `HashSet`, `add`, `remove` and `contains` take $O(1)$ time. `HashSet` is not thread safe.

### CopyOnWriteArraySet
Implementation uses `CopyOnWriteArrayList` internally therefore performance characteristic match that of an `ArrayList`. `add`, `remove` and `contains` take $O(n)$ time. The advantage over `HashSet` is that this variant provides thread safety - read operation doesn't need locking and is very fast. Write operation requires locks and is therefore expensive.

`CopyOnWriteArraySet` should be used when set size is relatively small, read operations are far more frequent than writes and thread safety is required. Its iterator provides a snapshot of how the set was when the `Iterator` was constructed. No iteration required while list traversal.

### EnumSet
A set that can be used to store subset of an enum members. The maximum number of elements is therefore the number of enum instance specified.

In [6]:
enum Planets { MERCURY, VENUS, EARTH, MARS, JUPITER, SATURN, NEPTUNE, URANUS }

EnumSet.of(Planets.MERCURY, Planets.MARS);    // of() creates modifiable collection in this instance

[MERCURY, MARS]

In [7]:
EnumSet.range(Planets.JUPITER, Planets.NEPTUNE);

[JUPITER, SATURN, NEPTUNE]

In [8]:
EnumSet.allOf(Planets.class);

[MERCURY, VENUS, EARTH, MARS, JUPITER, SATURN, NEPTUNE, URANUS]

Operations `add`, `remove` and `contains` all take $O(1)$ time. A simplistic implementation is given below, Java's implementation is more sophisticated and does bitwise operations on a long.

In [ ]:
class SimpleEnumSet<T extends Enum<T>> {
    private final boolean[] elements;
    
    public ESet(Class<T> enumClass) {
        elements = new boolean[enumClass.getEnumConstants().length];
    }

    public boolean add(T element) {
        int pos = element.ordinal();
        if (elements[pos]) {
            return false;
        } else {
            elements[pos] = true;
            return true;
        }
    }

    public boolean contains(T element) {
        return elements[element.ordinal()];
    }

    public boolean remove(T element) {
        int pos = element.ordinal();
        if (elements[pos]) {
            elements[pos] = false;
            return true;
        } else {
            return false;
        }
    }
}

### LinkedHashSet
A set that maintains insertion order on iteration:

In [9]:
LinkedHashSet<String> states = new LinkedHashSet<>();
states.add("California"); states.add("Nevada"); states.add("Texas"); states.add("Vermont");
states;

[California, Nevada, Texas, Vermont]

In [11]:
states.addLast("Vermont");
states;

[California, Nevada, Texas, Vermont]

Like `HashSet`, `LinkedHashSet` also has an underlying hash map.. Iterating is faster than `HashSet` though, since elements are all linked together.

### NavigableSet
Provide gurantee that its `Iterator` would traverse the set in sorted order.

In [12]:
record Car(String make, float bhp) {}

NavigableSet<Car> cars = new TreeSet<>((c1, c2) -> Float.compare(c2.bhp(), c1.bhp()));
cars.add(new Car("Veyron", 1001.0f));
cars.add(new Car("911 Turbo", 641.0f));
cars.add(new Car("918 Spyder", 887.0f));
cars;

[Car[make=Veyron, bhp=1001.0], Car[make=918 Spyder, bhp=887.0], Car[make=911 Turbo, bhp=641.0]]

In [13]:
// Remove and return elements
System.out.println(cars.pollFirst());
System.out.println(cars.pollLast());
cars;

Car[make=Veyron, bhp=1001.0]
Car[make=911 Turbo, bhp=641.0]


[Car[make=918 Spyder, bhp=887.0]]

If a `Comparator` is not provides, it is assumed that elements are instance of `Comparable`.

`SortedSet`, the parent of `NavigableSet` provides certain methods that establish view into the collection:

In [16]:
SortedSet<String> trees = new TreeSet<>();
trees.add("Oak"); trees.add("Maple"); trees.add("Eucalyptus"); trees.add("Banyan");

System.out.println(trees.subSet("Banyan", "Maple"));
System.out.println(trees.headSet("Maple"));
System.out.println(trees.tailSet("Eucalyptus"));

[Banyan, Eucalyptus]
[Banyan, Eucalyptus]
[Eucalyptus, Maple, Oak]


Adding to the view may pass or may fail with "IllegalArgumentException: key out of range" exception. Changing the origianl list also updates the view automatically.

**`TreeSet`** is the implementation of `NavigableSet` provided by Java. `TreeSet` internally uses *red-black tree*. `add`, `contains` and `remove` are executed in $O(\log{n})$ time.

### ConcurrentSkipList
Backed by [SkipList](https://en.wikipedia.org/wiki/Skip_list), this set provides thread safe and sorted set . Operations `add`, `remove` and `contains` take $O(\log{n})$ time.

## Queue
![Queue Hierarchy](./images/queue.png)

A queue is a container to hold elements for processing. Typically queues are FIFO, elements that arrive first leave first. However this is not mandatory, there are queue implementation based on element priority. Items are added to the *tail* of the queue and retreived from the *head*. Some queues may have a restriction on maximum number of elements it can hold - called as *bounded* queue.

The `Queue` interface defines the following operations:

In [ ]:
// Below methods return exception
boolean add(E e);  // exception if queue is full
E remove(); // exception if queue is empty
E element(); // exception if queue is empty

// The versions below do not throw exception
boolean offer(E e);  // can still throw other kind of exception like NPE
E poll();  // remove and get an element from the queue or null
E peek();  // get the head element or null

Since `peek` and `poll` return null to indicate empty queue, it is generally recommended not to store null in queues. Implementation like `ArrayDeque` in fact returns `NullPointerException` if we `offer(null)`. Since `Queue` implements `Collection` it does support traversal using an `Iterator`. However, queues are not designed for fast traversal and there is no guarantee that elements will be returned in proper sequence.

### PriorityQueue
Is a non-thread safe, non-blocking unbounded queue which stores elements in sorted order. `PriorityQueue` can return any arbitrary element if these elements have the same priority.

In [18]:
Queue<Integer> queue = new PriorityQueue<>();
queue.offer(5); queue.offer(2); queue.offer(7); queue.offer(4);
queue.poll(); // ascending order of priority

2

In [19]:
Queue<Integer> queue = new PriorityQueue<>((a, b) -> b - a);
queue.offer(5); queue.offer(2); queue.offer(7); queue.offer(4);
queue.poll();

7

`PriorityQueue` is implemented internally as a priority heap. Operations like `offer`, `add`, `poll` and `remove()` take $O(\log{n})$ time. Operation `remove(Object o)` and `contains` require $O(n)$ time. `peek` and `element` take $O(1)$ time. The iterator simply traverses over the underlying array, so it may not be in the priority order.

### ConcurrentLinkedQueue
Is a thread-safe, non-blocking, unbounded, FIFO ordered queue. It executes `offer` and `poll` in $O(1)$ time, but take $O(n)$ time for `size`.

### BlockingQueue
Queues of this type are mostly intended for producer consumer scenarios. As the name suggests, these queues have enqueue and dequeue methods that blocking til certain condition is met or support timeout.

In [ ]:
// In addition to usual exception returning method add and boolean returning version
// offer, blocking queues have methods that block
void put(E e) throws InterruptedException;  // blocks till space is available in queue
boolean offer(E e, long timeout, TimeUnit unit) throws InterruptedException;

E take() throws InterruptedException;
E poll(long timeout, TimeUnit unit) throws InterruptedException;

int drainTo(Collection<? super E> c); // atomically clear the queue into c
int drainTo(Collection<? super E> c, int n); // atomically clear max n number of elements from the queue into c

Blocking queue operations are thread safe, though some operations like `addAll`, `containsAll`, `retailAll` and `removeAll`'s thread safety can be implementation dependent. There are several implementations available:  
**LinkedBlockingQueue:** linked list based, FIFO ordering. `offer` and `poll` are executed in $O(1)$ time. `contains` require linked list traversal taking $O(n)$ time.  
**ArrayBlockingQueue:** FIFO ordered queue implemented using circular array. Complexity similar to `LinkedBlockingQueue`.  
**PriorityBlockingQueue:** thread-safe blocking version of `PriorityQueue` with similar performance characterisitics.

### Deque
![Deque](./images/deque.png)  
Are double-ended queues which allow element insertion and retrieval from either end. If element is added at one end and removed at another, then it acts like a FIFO queue. However, if element is added and removed from the same end, it behaves as a LIFO stack. It uses terms like first and last; first refers to head and last refers to tail. It has methods like:

In [ ]:
void addFirst(E e);  // inserts at head if enough space, else throws exception
void addLast(E e);   // inserts at tail if enough space, else throws exception

void offerFirst(E e);
void offerLast(E e); 

void push(E e);  // same as addFirst, named to reflect use of deque as stack

E peekFirst(); // retrieve and return head element
E peekLast();  // retrieve and return tail element

E pollFirst(); // remove and return head element
E pollLast();  // remove and return head element

// These throw exception for empty deque
E getFirst();
E getLast();
E removeFirst();
E removeLast();
E pop();

// Of the seven methods in SequencedCollection, six were promoted from Deque
// only the brlow method is newly introduced
Deque<E> reversed();

Implementation of `Deque` include:  
**ArrayDeque:** implemented using a circular array like `ArrayBlockingQueue`. It is the general purpose queue and deque implementation provided by Java. Other implementations are either thread safe (not the best performing in single threaded application) or based on linked list like `LinkedList`. Operations that insert to queue lead to `NullPointerException` if null is inserted:

In [20]:
Queue<Integer> arrayDeque = new ArrayDeque<>();
arrayDeque.add(null);  // same as addLast or offer

EvalException: null

**LinkedList:** is peculiar since it implements both `List` and `Queue`. Originally it only implemented `List`, but later it implemented `Queue` and then `Deque`.  
**BlockingDeque:** adds two new methods in addition to the methods it inherits from `BlockingQueue`:

In [ ]:
void putFirst(E e) throws InterruptedException;;
void putLast(E e) throws InterruptedException;;

E takeFirst() throws InterruptedException;
E takeLast() throws InterruptedException;

Java provides one implementation `LinkedBlockingDeque` with performance same as `LinkedBlockingQueue`. This data-structure is often used in case of work stealing load balancer.